# Validate Minian outputs

- Load
    - spatial map (A) -> maybe to associate with a projection of neuron fluorescence
    - temporal activity of detected neurons

- Validate units
- Extract and save relevant data for each selected unit
    - spatial location (x, y); no need for shape
    - Ca2+ trace
    - Ca2+ peak time and amplitude
    - Deconvolved spikes


## Load the required packages (and many more that are not useful)

In [ ]:
import numpy as np
import csv
from pathlib import Path
import xarray as xr
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
import itertools as itt
import os
import sys
import json
from scipy import signal
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

import holoviews as hv
import panel as pn
import param
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from holoviews import opts
from holoviews import Store
from holoviews.operation.datashader import shade
hv.extension('bokeh', 'matplotlib')
#from IPython.core.display import display
from IPython.display import display
from ipyfilechooser import FileChooser


import warnings
warnings.filterwarnings("ignore")
#%reset

In [ ]:
minian_path = os.path.join(os.path.abspath('..'),'minian')
print("The folder used for minian procedures is : {}".format(minian_path))

sys.path.append(minian_path)
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)

## Import the minian files

### Select the folder

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)



### Import spatial map, Ca2+ traces, deconvolved spikes

In [ ]:
minianversion = 'minian'
try: # tries to retrieve minianversion either from a previous run or from a previous notebook
    %store -r minianversion
except:
    print("the minian folder to use was not defined in store")
    minianversion = 'minian' #'minianAB' # or 'minian_intermediate'
    %store minianversion

folderMouse = Path(os.path.join(dpath,minianversion))
print(folderMouse)
minian_ds = open_minian(folderMouse)

A = minian_ds['A']
C = minian_ds['C']
S = minian_ds['S']

B = A['unit_id']
series = B.to_series()
D = series.count()

idloc = A.idxmax("unit_id")
Hmax = A.idxmax("height")
Hmax2 = Hmax.max("width")

Wmax = A.idxmax("width")
Wmax2 = Wmax.max("height")
coord1 = Wmax2.to_series()
coord2 = Hmax2.to_series()

a = pd.concat([coord1,coord2], axis=1)
unit = len(a)
print("{} units have been found".format(unit))

## Plot either all cells or just the one of interest

### Define the actions triggered by drop/keep buttons

In [5]:
unit_to_keep=a.index.tolist()   

In [6]:
# Set up selector object
discrete_slider = pn.widgets.DiscreteSlider(
    name="Choose unit", 
    options=[i for i in a.index],
    value=a.index[0]
)

drop_unit_button = pn.widgets.Button(name='Drop this unit! >', button_type='primary')
keep_unit_button = pn.widgets.Button(name='Keep this unit!')
next_unit_button = pn.widgets.Button(name='Next unit > ', button_type='primary')
previous_unit_button = pn.widgets.Button(name='< Previous unit')

text_input = pn.widgets.TextInput(value='', width=300)

# Define a callback function for the button
def dropunit_callback(event):
    selected_value = discrete_slider.value
    text=f'Unit n°{selected_value} dropped!'
    number = selected_value
    if number in unit_to_keep:
        unit_to_keep.remove(number)
    nextunit_callback(event)
    text_input.value = text

# Define a callback function for the button
def keepunit_callback(event):
    selected_value = discrete_slider.value
    text=f'Unit n°{selected_value} kept!'
    number = selected_value
    if number not in unit_to_keep:
        unit_to_keep.append(number)
        unit_to_keep.sort()
    text_input.value = text

# Define a callback function for the button
def nextunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    nextunitvalue=a.index[position + 1] if position+2<=len(a) else a.index[0]
    discrete_slider.value = nextunitvalue
    
# Define a callback function for the button
def previousunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    previousunitvalue=a.index [position - 1]
    discrete_slider.value = previousunitvalue

drop_unit_button.on_click(dropunit_callback)
keep_unit_button.on_click(keepunit_callback)
next_unit_button.on_click(nextunit_callback)
previous_unit_button.on_click(previousunit_callback)

In [7]:
# Define interactivity
@pn.depends(indexes=discrete_slider)
def calciumtrace(indexes):
    index = indexes
    position = np.where(a.index == index)[0]
    position = position[0]
    return hv.Curve((C[position, :]), label=f'Unit n° {index}').opts(ylim=(-0.5, 20))

@pn.depends(indexes=discrete_slider)
def circlepath(indexes):
    index = indexes
    radius = 15
    num_points=100
    theta = np.linspace(0, 2*np.pi, num_points)
    position = np.where(a.index == index)[0]
    position = position[0]
    return hv.Path((a.iloc[position, 0] + radius * np.cos(theta), a.iloc[position, 1] + radius * np.sin(theta)), group='keep').opts(ylim=(0, 600), xlim=(0, 600), line_color='red', line_width=3) #

#@pn.depends(indexes=discrete_slider)
#def imgZoom(indexes):
#    index = indexes
#    position = np.where(a.index == index)[0]
#    position = position[0]
#    xx1=int(a.iloc[position, 0]-20)
#    xx2=int(a.iloc[position, 0]+20)
#    yy1=int(a.iloc[position, 1]-20)
#    yy2=int(a.iloc[position, 1]+20)   
#    return hv.Image(A.max("unit_id")[yy1:yy2,xx1:xx2], kdims=["width", "height"]).opts(ylim=(xx1, xx2), xlim=(yy1, yy2), colorbar=False, invert_yaxis=False,cmap="Viridis", shared_axes=False)

### Plot the spatial map for all cells + interactive Ca2+ trace

In [ ]:
output_size = 180
hv.output(size=int(output_size))

image = hv.Image(
    A.max("unit_id").compute().astype(np.float32).rename("A"),
    kdims=["width", "height"],
).opts(colorbar=False, invert_yaxis=False,cmap="Viridis")

#labels = hv.Labels([(a.iloc[i, 0], a.iloc[i, 1], a.index[i]) for i in range(len(a))]).opts(text_color='white', text_font_size='7pt') 

#layout = pn.Row(image * labels * hv.DynamicMap(circlepath), hv.DynamicMap(calciumtrace), # hv.DynamicMap(imgZoom)* hv.DynamicMap(circlepath),
#            pn.Column(discrete_slider, pn.Row(previous_unit_button, next_unit_button), pn.Row(keep_unit_button, drop_unit_button), text_input       
#                    )
#                    )

layout = pn.Row(image * hv.DynamicMap(circlepath), hv.DynamicMap(calciumtrace), # hv.DynamicMap(imgZoom)* hv.DynamicMap(circlepath),
            pn.Column(discrete_slider, pn.Row(previous_unit_button, next_unit_button), pn.Row(keep_unit_button, drop_unit_button), text_input       
                    )
                    )

display(layout)


Feel unit to keep

In [ ]:
print("The units kept are:")
print(unit_to_keep)

all_units=a.index.tolist()
unit_to_drop = [e for e in all_units if e not in unit_to_keep]
print("The units dropped are:")
print(unit_to_drop)


Remove dropped units

In [ ]:
copyB = list(B.copy())
for i in range(len(unit_to_drop)):
    elem = unit_to_drop[i]
    copyB.remove(elem) # IF CELL ID 
unit_to_keep = copyB

A_upd = A.loc[unit_to_keep,:,:]
C_upd = C.loc[unit_to_keep,:]
S_upd = S.loc[unit_to_keep,:]

TodropFile = folderMouse / f'TodropFileAB.json'

with open(TodropFile, 'w') as f:
    json.dump(unit_to_drop, f, indent=2) 